## Bokeh 

In [1]:
!pip install -U ipykernel
!pip install holoviews

Requirement already up-to-date: ipykernel in /usr/local/lib/python3.6/dist-packages (5.2.1)


In [2]:
# !pip install -U ipykernel
# !pip install holoviews
import bokeh.sampledata
bokeh.sampledata.download()

import holoviews as hv
from holoviews import dim
from bokeh.sampledata.airport_routes import routes, airports

hv.extension('matplotlib')
hv.output(fig='svg', size=300)

Output hidden; open in https://colab.research.google.com to view.

In [3]:
routes.head()

,Airline,AirlineID,Source,SourceID,Destination,DestinationID,Codeshare,Stops,Equipment
0,2O,146,ADQ,3531,KLN,7162,NaN,0,BNI
1,2O,146,KLN,7162,KYK,7161,NaN,0,BNI
2,3E,10739,BRL,5726,ORD,3830,NaN,0,CNC
3,3E,10739,BRL,5726,STL,3678,NaN,0,CNC
4,3E,10739,DEC,4042,ORD,3830,NaN,0,CNC


In [4]:
routes.describe()

,SourceID,DestinationID,Stops
count,10507.000000,10507.000000,10507.000000
mean,3850.197868,3851.889597,0.000571
std,669.717606,672.642840,0.023891
min,3411.000000,3411.000000,0.000000
25%,3577.000000,3577.000000,0.000000
50%,3697.000000,3697.000000,0.000000
75%,3845.000000,3849.000000,0.000000
max,9744.000000,9744.000000,1.000000


In [0]:
# Count the routes between Airports
route_counts = routes.groupby(['SourceID', 'DestinationID']).Stops.count().reset_index()
nodes = hv.Dataset(airports, 'AirportID', 'City')
chord = hv.Chord((route_counts, nodes), ['SourceID', 'DestinationID'], ['Stops'])

# Select the 20 busiest airports
busiest = list(routes.groupby('SourceID').count().sort_values('Stops').iloc[-20:].index.values)
busiest_airports = chord.select(AirportID=busiest, selection_mode='nodes')

In [6]:
busiest_airports.opts(
    cmap='Category20', labels='City',
    edge_color=dim('SourceID').astype(str),
    node_color=dim('AirportID').astype(str))

:Chord   [SourceID,DestinationID]   (Stops)

In [15]:
%env HV_DOC_HTML=true
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.plotting import figure
from bokeh.sampledata.autompg import autompg_clean as df
from bokeh.transform import factor_cmap

df.cyl = df.cyl.astype(str)
df.yr = df.yr.astype(str)

group = df.groupby(by=['cyl', 'mfr'])
source = ColumnDataSource(group)

p = figure(plot_width=800, plot_height=300, title="Mean MPG by # Cylinders and Manufacturer",
           x_range=group, toolbar_location=None, tools="")

p.xgrid.grid_line_color = None
p.xaxis.axis_label = "Manufacturer grouped by # Cylinders"
p.xaxis.major_label_orientation = 1.2

index_cmap = factor_cmap('cyl_mfr', palette=['#2b83ba', '#abdda4', '#ffffbf', '#fdae61', '#d7191c'], 
                         factors=sorted(df.cyl.unique()), end=1)

p.vbar(x='cyl_mfr', top='mpg_mean', width=1, source=source,
       line_color="white", fill_color=index_cmap, 
       hover_line_color="darkgrey", hover_fill_color=index_cmap)

p.add_tools(HoverTool(tooltips=[("MPG", "@mpg_mean"), ("Cyl, Mfr", "@cyl_mfr")]))

show(p)

env: HV_DOC_HTML=true


In [8]:
# Create and deploy interactive data applications

from IPython.display import IFrame
IFrame('https://demo.bokeh.org/sliders', width=900, height=500)

In [38]:
#!pip install -q holoviews 
%env HV_DOC_HTML=true

import numpy as np
import pandas as pd
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from holoviews import opts

hv.extension('bokeh')

output_notebook()
plot = figure(y_axis_label=("U.S. Postage Rates (1999-2015)"), plot_width=300, plot_height=300)
plot.step(x=list(range(0, 17)), y=[.20, .20, .21, .23, .23, .23, .23, .24, .26, .27,
            .28, .28, .29, .32, .33, .34, .35], color="#FB8072")
show(plot)

Output hidden; open in https://colab.research.google.com to view.

In [39]:
diseases= pd.read_csv('https://frenzy86.s3.eu-west-2.amazonaws.com/fav/diseases.csv')
diseases

,Year,Week,State,measles,pertussis
0,1928,1,Alabama,3.67,NaN
1,1928,2,Alabama,6.25,NaN
2,1928,3,Alabama,7.95,NaN
3,1928,4,Alabama,12.58,NaN
4,1928,5,Alabama,8.03,NaN
...,...,...,...,...,...
222763,2011,48,Wyoming,NaN,0.0
222764,2011,49,Wyoming,NaN,0.0
222765,2011,50,Wyoming,NaN,0.0
222766,2011,51,Wyoming,NaN,0.0


In [40]:
vdims = [('measles', 'Measles Incidence'), ('pertussis', 'Pertussis Incidence')]
ds = hv.Dataset(diseases, ['Year', 'State'], vdims)

ds = ds.aggregate(function=np.mean)
ds

:Dataset   [Year,State]   (measles,pertussis)

In [41]:
layout = (ds.to(hv.Curve, 'Year', 'measles') + ds.to(hv.Curve, 'Year', 'pertussis')).cols(1)
layout.opts(
    opts.Curve(width=600, height=250, framewise=True))

show(hv.render(layout))

In [42]:
states = ['New York', 'New Jersey', 'California', 'Texas']
bars = ds.select(State=states, Year=(1980, 1990)).to(hv.Bars, ['Year', 'State'], 'measles').sort()
bars.opts(
    opts.Bars(width=800, height=400, tools=['hover'], xrotation=90, show_legend=False))

show(hv.render(bars))

In [43]:
grouped = ds.select( State=states, Year=(1930, 2005)).to(hv.Curve, 'Year', 'measles')
gridspace = grouped.grid('State')
gridspace.opts(
    opts.Curve(width=200, color='indianred'))

show(hv.render(gridspace))

In [44]:
ndoverlay = grouped.overlay('State')
ndoverlay.opts(
    opts.Curve(width=600, color=hv.Cycle(values=['indianred', 'slateblue', 'lightseagreen', 'coral'])))

show(hv.render(ndoverlay))

In [45]:
import pandas as pd
import holoviews as hv

hv.extension('bokeh')

Output hidden; open in https://colab.research.google.com to view.

In [46]:
edges = pd.read_csv('https://frenzy86.s3.eu-west-2.amazonaws.com/fav/energy.csv')
edges.head()

,source,target,value
0,Agricultural 'waste',Bio-conversion,124.729
1,Bio-conversion,Liquid,0.597
2,Bio-conversion,Losses,26.862
3,Bio-conversion,Solid,280.322
4,Bio-conversion,Gas,81.144


In [47]:
sankey = hv.Sankey(edges, label='Energy Diagram')
sankey.opts(label_position='left', edge_color='target', node_color='index', cmap='tab20')

show(hv.render(sankey))

In [0]:
url = 'https://raw.githubusercontent.com/blmoore/blogR/master/data/measles_incidence.csv'
data = pd.read_csv(url, skiprows=2, na_values='-')

yearly_data = data.drop('WEEK', axis=1).groupby('YEAR').sum().reset_index()
measles = pd.melt(yearly_data, id_vars=['YEAR'], var_name='State', value_name='Incidence')

heatmap = hv.HeatMap(measles, label='Measles Incidence')
aggregate = hv.Dataset(heatmap).aggregate('YEAR', np.mean, np.std)

vline = hv.VLine(1963)
marker = hv.Text(1964, 800, 'Vaccine introduction', halign='left')

agg = hv.ErrorBars(aggregate) * hv.Curve(aggregate)

In [49]:
overlay = (heatmap + agg * vline * marker).cols(1)
overlay.opts(
    opts.HeatMap(width=900, height=500, tools=['hover'], logz=True, 
                   invert_yaxis=True, labelled=[], toolbar='above', xaxis=None),
    opts.VLine(line_color='black'),
    opts.Overlay(width=900, height=200, show_title=False, xrotation=90))
show(hv.render(overlay))

In [50]:
hv.extension('matplotlib')
hv.output(fig='svg')


(heatmap + agg * vline * marker).opts(
    opts.HeatMap(aspect=1.4,invert_yaxis=True, show_values=False, show_frame=False,
                 labelled=[], xaxis=None, logz=True),
    opts.Layout(aspect_weight=1, fig_size=300, vspace=0, sublabel_format=None),
    opts.Overlay(aspect=3, show_title=False, bgcolor='white', 
                 show_legend=False, show_frame=False, xrotation=90),
    opts.VLine(color='black', ylim=(0, 1050))).cols(1)

Output hidden; open in https://colab.research.google.com to view.

# Form

In [0]:
#@title String fields

text = 'value' #@param {type:"string"}
dropdown = '1st option' #@param ["1st option", "2nd option", "3rd option"]
text_and_dropdown = 'value' #@param ["1st option", "2nd option", "3rd option"] {allow-input: true}

print(text)
print(dropdown)
print(text_and_dropdown)

In [0]:
#@title Raw fields

raw_input = None #@param {type:"raw"}
raw_dropdown = raw_input #@param [1, "raw_input", "False", "'string'"] {type:"raw"}

print(raw_input)
print(raw_dropdown)

In [0]:
#@title Date fields
date_input = '2018-03-22' #@param {type:"date"}

print(date_input)

In [0]:
#@title Number fields
number_input = 10.0 #@param {type:"number"}
number_slider = 0 #@param {type:"slider", min:-1, max:1, step:0.1}

integer_input = 10 #@param {type:"integer"}
integer_slider = 1 #@param {type:"slider", min:0, max:100, step:1}

print(number_input)
print(number_slider)

print(integer_input)
print(integer_slider)

In [0]:
#@title Boolean fields
boolean_checkbox = True #@param {type:"boolean"}
boolean_dropdown = True #@param ["False", "True"] {type:"raw"}

print(boolean_checkbox)
print(boolean_dropdown)

In [0]:
#@title ## Markdown
#@markdown You can also include Markdown in forms.

#@markdown ---
#@markdown ### Enter a file path:
file_path = "" #@param {type:"string"}
#@markdown ---

In [0]:
#@title Double click on this line in the code cell. { display-mode: "form" }

option1 = 'B' #@param ["A", "B", "C"]
print('You selected', option1)

In [0]:
#@title After running this cell manually, it will auto-run if you change the selected value. { run: "auto" }

option2 = "A" #@param ["A", "B", "C"]
print('You selected', option2)

# Utilizzo Form Negli esercizi

In [0]:
def area_quadrato(x):
    area=x**2
    return area

In [0]:
lato_int = 50 #@param {type:"slider", min:0, max:100, step:1}

In [58]:
area_quadrato(lato_int)

2500

In [0]:
lato_float = 58.51 #@param {type:"slider", min:0, max:100, step:0.01}

In [60]:
area_quadrato(lato_float)

3423.4201

In [0]:
### numeri negativi

In [0]:
neg = -58.92 #@param {type:"slider", min:-100, max:100, step:0.01}